In [1]:
import pandas as pd
import numpy as np
import os
import quandl
import datetime
import backtrader as bt
import backtrader.feeds as btfeeds

quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')

In [3]:
import pandas_datareader as pdr
from datetime import datetime

appl = pdr.get_data_yahoo(symbols='AAPL', start=datetime(2000, 1, 1), end=datetime(2000, 12, 31))
print(appl.head())
print(appl.tail())

                High       Low      Open     Close       Volume  Adj Close
Date                                                                      
2000-01-03  4.017857  3.631696  3.745536  3.997768  133949200.0   2.677157
2000-01-04  3.950893  3.613839  3.866071  3.660714  128094400.0   2.451444
2000-01-05  3.948661  3.678571  3.705357  3.714286  194580400.0   2.487319
2000-01-06  3.821429  3.392857  3.790179  3.392857  191993200.0   2.272070
2000-01-07  3.607143  3.410714  3.446429  3.553571  115183600.0   2.379695
                High       Low      Open     Close       Volume  Adj Close
Date                                                                      
2000-12-22  1.071429  1.008929  1.008929  1.071429   79513000.0   0.717496
2000-12-26  1.071429  1.017857  1.062500  1.049107   54203800.0   0.702548
2000-12-27  1.058036  1.013393  1.024550  1.058036   81366600.0   0.708527
2000-12-28  1.066964  1.022321  1.026786  1.058036   76294400.0   0.708527
2000-12-29  1.071429  1.0

In [15]:
data = btfeeds.YahooFinanceData(
    dataname='AAPL',

    fromdate=datetime(2000, 1, 1),
    todate=datetime(2000, 12, 31),

    datetime=0,
    high=1,
    low=2,
    open=3,
    close=4,
    volume=5,
    openinterest=-1
)

In [24]:
columns = ['date', 'high', 'low', 'open', 'close', 'volume']
df = quandl.get_table('SHARADAR/SEP', ticker='AAPL', qopts={"columns":columns}, 
                      date={'gte': datetime.datetime(2000, 1, 1), 'lte': datetime.datetime(2000, 12, 31)})
df.set_index('date', inplace=True)
print(df.head())
print(df.tail())

             high    low   open  close       volume
date                                               
2000-12-29  1.071  1.036  1.049  1.063  157584000.0
2000-12-28  1.067  1.022  1.027  1.058   76294400.0
2000-12-27  1.058  1.014  1.024  1.058   81366600.0
2000-12-26  1.071  1.018  1.063  1.049   54203800.0
2000-12-22  1.071  1.009  1.009  1.071   79513000.0
             high    low   open  close       volume
date                                               
2000-01-07  3.607  3.411  3.446  3.554  115183600.0
2000-01-06  3.821  3.393  3.790  3.393  191993200.0
2000-01-05  3.949  3.679  3.705  3.714  194580400.0
2000-01-04  3.951  3.614  3.866  3.661  128094400.0
2000-01-03  4.018  3.632  3.745  3.998  133949200.0


In [32]:
data = btfeeds.Quandl(
    dataname='AAPL',
    apikey=os.getenv('QUANDL_APY_KEY'),
    dataset='SHARADAR/SEP',
    reverse=False,

    fromdate=datetime.datetime(2000, 1, 1),
    todate=datetime.datetime(2000, 12, 31),

    datetime=1,
    high=3,
    low=4,
    open=2,
    close=5,
    volume=6,
    openinterest=-1
)

In [33]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

In [34]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


AttributeError: 'NoneType' object has no attribute 'close'